# Clean up Amazon Personalize resources
This notebook deletes the set of solutions (along with their corresponding
solution versions) and campaigns from Amazon Personalize. It also deletes the event tracker.

Note that the datasets and the dataset group are left in place, as they may be reused. The monthly charges for ongoing storage of imported data, and the data we uploaded to S3, 
are relatively inexpensive.

In [ ]:
account_num = '<your-account-num>'

In [ ]:
import json
import boto3
import time

file_name = 'DEMO-movie-lens-100k.csv'

region = boto3.Session().region_name # or replace with your preferred region
print(region)

dataset_group_name = 'sf-loft-recs-dataset-group'
dataset_group_arn = 'arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, 
                                                                        account_num, 
                                                                        dataset_group_name)

MAX_WAIT_TIME = time.time() + 3*60*60 # 3 hours

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Delete campaigns

In [ ]:
def delete_campaign(c):
    try:
        personalize.delete_campaign(campaignArn = c)
    except Exception as e:
        pass
        
def delete_campaigns():
    delete_campaign('arn:aws:personalize:{}:{}:campaign/similar-items-campaign'.format(region, account_num))
    delete_campaign('arn:aws:personalize:{}:{}:campaign/personal-rankings-campaign'.format(region, account_num))
    delete_campaign('arn:aws:personalize:{}:{}:campaign/sf-loft-lab-recs-campaign'.format(region, account_num))

In [ ]:
delete_campaigns()

In [ ]:
print('Wating for campaign deletion to complete...')
time.sleep(30)

## Delete solutions

In [ ]:
def delete_solution(s):
    try:
        personalize.delete_solution(solutionArn = s)
    except Exception as e:
        pass
    
def delete_solutions():
    delete_solution('arn:aws:personalize:{}:{}:solution/rankings-solution'.format(region, account_num))
    delete_solution('arn:aws:personalize:{}:{}:solution/similar-items-solutions'.format(region, account_num))
    delete_solution('arn:aws:personalize:{}:{}:solution/sf-loft-lab-recs-solution'.format(region, account_num))    

In [ ]:
delete_solutions()

In [ ]:
print('Waiting for solution deletion to complete...')
time.sleep(60)

## Delete dataset

In [ ]:
def delete_dataset(d):
    try:
        print('Deleting {}'.format(d))
        personalize.delete_dataset(datasetArn=d)
    except Exception as e:
        print(e)
        pass

def delete_datasets():
    delete_dataset('arn:aws:personalize:{}:{}:dataset/{}/EVENT_INTERACTIONS'.format(region, 
                                                                                    account_num, 
                                                                                    dataset_group_name))
    delete_dataset('arn:aws:personalize:{}:{}:dataset/{}/INTERACTIONS'.format(region, account_num, dataset_group_name))
    delete_dataset('arn:aws:personalize:{}:{}:dataset/{}/USERS'.format(region, account_num, dataset_group_name))
    delete_dataset('arn:aws:personalize:{}:{}:dataset/{}/ITEMS'.format(region, account_num, dataset_group_name))

In [ ]:
delete_datasets()

In [ ]:
print('Waiting for dataset deletion to complete...')
time.sleep(30)

## Delete schema

In [ ]:
def delete_schema(s):
    try:
        print('Deleting {}'.format(s))
        personalize.delete_schema(schemaArn=s)
    except Exception as e:
        print(e)
        pass

def delete_schemas():
    delete_schema('arn:aws:personalize:{}:{}:schema/{}'.format(region, account_num, 'sf-loft-lab-recs-schema'))

In [ ]:
delete_schemas()

## Delete event trackers and dataset group

In [ ]:
resp = personalize.list_event_trackers()
trackers = resp['eventTrackers']
for i in range(len(trackers)):
    print('name: {}, arn: {}'.format(trackers[i]['name'], trackers[i]['eventTrackerArn']))
    personalize.delete_event_tracker(eventTrackerArn=trackers[i]['eventTrackerArn'])

In [ ]:
print('Waiting for event tracker deletion to complete...')
time.sleep(30)

In [ ]:
try:
    personalize.delete_dataset_group(datasetGroupArn='arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, 
                                                                                                         account_num, 
                                                                                                         dataset_group_name))
except Exception as e:
    print('Could not delete dataset group, exception: \n{}'.format(e))
    pass